# Capstone Project — The Battle of Neighborhoods- The London Insight

### Setting up a shopping mall in London Neigbourhood, United Kingdom.

## 1 Introduction

London is the capital and largest city of England and the United Kingdom.There really isn’t any shopping experience quite like London. London offers several distinct shopping areas, streets, outlets and malls
The population of London has grown considerably over the last decades and for many visiting, shopping mall is a perfect way to relax and enjoy themselves.
This present a good businness opporturnity for a property development company that wants to invest on opening new shopping mall.

## Business Problem

The purpose of this capstone project is to help prospective property developers explore the best locations arround london Boroughs. it will help them make 
efficient decision on selecting best neighborhood to open new shopping mall.


## Methodology

For this project we are going to use the K-means Clustering algorithm to cluster the neighborhoods and focused on geospatial analysis of the London to 
understand which would be the best neighborhoods to open a new mall.
Foursquare API to provide different venue categories.
The business question is: In London, if a property developer is looking to open a new shopping mall, where would you recommend that they open it?

## Data Section

Data Link:https://en.wikipedia.org/wiki/List_of_places_in_London

List of the 32 Bororghs in London and there neighborhoods.
Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map and also to get the venue data, particularly data related to shopping malls. 

In [1]:
import pandas as pd

The wikipedia data has already been downloaded, cleaned and saved as london-borough.csv

In [2]:
df = pd.read_csv('london-borough.csv', usecols=['Area name','Inner/ Outer London','GLA Population Estimate 2016'])

In [3]:
df.head()

,Area name,Inner/ Outer London,GLA Population Estimate 2016
0,NaN,NaN,NaN
1,City of London,Inner London,"8,548"
2,Barking and Dagenham,Outer London,"205,773"
3,Barnet,Outer London,"385,108"
4,Bexley,Outer London,"243,303"


In [4]:
df.tail(10)

,Area name,Inner/ Outer London,GLA Population Estimate 2016
30,Tower Hamlets,Inner London,"297,805"
31,Waltham Forest,Outer London,"274,139"
32,Wandsworth,Inner London,"318,253"
33,Westminster,Inner London,"239,862"
34,Inner London,NaN,"3,494,269"
35,Outer London,NaN,"5,256,484"
36,London,NaN,"8,750,753"
37,England,NaN,"54,316,618"
38,United Kingdom,NaN,"64,596,752"
39,National comparator,NaN,"54,316,618"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Area name                     39 non-null     object
 1   Inner/ Outer London           33 non-null     object
 2   GLA Population Estimate 2016  39 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB


Here we drop the rows that we do not need

In [6]:
df.dropna(axis=0, how='any', inplace=True)

In [7]:
df.head()

,Area name,Inner/ Outer London,GLA Population Estimate 2016
1,City of London,Inner London,"8,548"
2,Barking and Dagenham,Outer London,"205,773"
3,Barnet,Outer London,"385,108"
4,Bexley,Outer London,"243,303"
5,Brent,Outer London,"328,568"


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df.shape

(33, 3)

In [10]:
# Rename the dataframe columns
df.rename(columns={'Area name':'BoroughName', 'Inner/ Outer London':'Inner_Outer_London', 'GLA Population Estimate 2016':'Population'}, inplace=True)

In [11]:
df.head()

,BoroughName,Inner_Outer_London,Population
0,City of London,Inner London,"8,548"
1,Barking and Dagenham,Outer London,"205,773"
2,Barnet,Outer London,"385,108"
3,Bexley,Outer London,"243,303"
4,Brent,Outer London,"328,568"


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   BoroughName         33 non-null     object
 1   Inner_Outer_London  33 non-null     object
 2   Population          33 non-null     object
dtypes: object(3)
memory usage: 920.0+ bytes


## Import the necessary Libraries

In [13]:
import numpy as np # library to handle data in a vectorized manner

#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Drop the columns that are not needed to extract the list of BoroughName data.

In [14]:
# Drop columns not required
df = df.drop(columns=['Inner_Outer_London', 'Population',], axis=1)
df.head()

,BoroughName
0,City of London
1,Barking and Dagenham
2,Barnet
3,Bexley
4,Brent


In [15]:
df.shape

(33, 1)

In [16]:
df.tail()

,BoroughName
28,Sutton
29,Tower Hamlets
30,Waltham Forest
31,Wandsworth
32,Westminster


#### Get the geo-coordinates of London city using the geopy library of python

In [17]:
address = 'London, UK'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


#### So we proceed to applying it to our dataframe df

In [18]:
latitude = []
longitude = []
Borough = df['BoroughName']
for i in range(len(Borough)):
    address = str(Borough[i] + ', England')
    geolocator = Nominatim(user_agent="london_explorer")
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    location = geolocator.geocode(address)
    try: 
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append("Not Found")
        longitude.append("Not Found")
    
print('Done!')

Done!


#### Then we proceed to store the location data — latitude and longitude as follows. 

In [19]:
df_coord = df
df_coord['Latitude'] = latitude
df_coord['Longitude'] = longitude
df_coord.head()

,BoroughName,Latitude,Longitude
0,City of London,51.515618,-0.091998
1,Barking and Dagenham,51.554117,0.150504
2,Barnet,51.653090,-0.200226
3,Bexley,51.441679,0.150488
4,Brent,51.442189,0.236476


### Create a map of London city with all the boroughs superimposed on it.

In [20]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[51.5073219, -0.1276474], zoom_start=10)

# add markers to map
for lat, lng, boroughName in zip(df_coord['Latitude'], df_coord['Longitude'], df_coord['BoroughName']):
    label = '{}'.format(boroughName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### let's simplify the above map and segment and cluster only the neighborhoods in Newham and create a new dataframe of the new_neig.

In [21]:
Borough = ['Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham']



Neighborhood = ['Beckton','Canning Town','Cyprus','East Ham','East Village','Forest Gate',
'Manor Park','Maryland','North Woolwich','Plashet',
'Silvertown','Stratford','Upton Park','Wallend','West Ham']


Latitude = ['','','','','','','','','','','','','','','']
Longitude = ['','','','','','','','','','','','','','','']

df_neigh = {'Neighborhood': Neighborhood,'Borough':Borough,'Latitude': Latitude,'Longitude':Longitude}
new_neig = pd.DataFrame(data=df_neigh, columns=['Neighborhood', 'Borough', 'Latitude', 'Longitude'], index=None)

new_neig

,Neighborhood,Borough,Latitude,Longitude
0,Beckton,Newham,,
1,Canning Town,Newham,,
2,Cyprus,Newham,,
3,East Ham,Newham,,
4,East Village,Newham,,
5,Forest Gate,Newham,,
6,Manor Park,Newham,,
7,Maryland,Newham,,
8,North Woolwich,Newham,,
9,Plashet,Newham,,


#### Get the geo-coordinates of Newham Borough using the geopy library 

In [22]:
address = 'Newham, London'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Newham are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Newham are 51.52999955, 0.02931796029382208.


#### we proceed to applying it to our dataframe df_coord

In [23]:
latitude = []
longitude = []
Borough = new_neig['Borough']
Neighborhood = new_neig['Neighborhood']
for i in range(len(Neighborhood)):
    address = str(Neighborhood[i] + ', England')
    geolocator = Nominatim(user_agent="explorer")
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    location = geolocator.geocode(address)
    try: 
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append("Not Found")
        longitude.append("Not Found")
    
print('Done!')

Done!


#### Then we proceed to store the location data — latitude and longitude as follows.

In [24]:
df_coord = new_neig
df_coord['Latitude'] = latitude
df_coord['Longitude'] = longitude
df_coord.head(14)

,Neighborhood,Borough,Latitude,Longitude
0,Beckton,Newham,51.516080,0.059426
1,Canning Town,Newham,51.513989,0.008299
2,Cyprus,Newham,51.508478,0.063969
3,East Ham,Newham,51.532963,0.055320
4,East Village,Newham,51.548108,-0.009177
5,Forest Gate,Newham,51.549524,0.024925
6,Manor Park,Newham,51.426949,-0.135507
7,Maryland,Newham,51.546053,0.005922
8,North Woolwich,Newham,51.500407,0.064154
9,Plashet,Newham,51.540008,0.039274


Let's get the geographical coordinates of Newham

In [25]:
locations = df_coord[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[51.5460532, 0.0059223]

As we did with all of London, let's visualizat Newham the neighborhoods in it

In [26]:
map = folium.Map(location=[51.5460532, 0.0059223], zoom_start=11)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_coord['Neighborhood'][point]).add_to(map)
map

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [27]:
# @hidden_cell
CLIENT_ID = 'VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO' # your Foursquare ID
CLIENT_SECRET = 'VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO
CLIENT_SECRET:VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM


### We can now explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [28]:
df_coord.loc[0, 'Neighborhood']

'Beckton'

Get the neighborhood's latitude and longitude values

In [29]:
neighborhood_latitude = df_coord.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_coord.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_coord.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beckton are 51.5160797, 0.0594257.


#### Now, let's get the top 100 venues that are in Beckton within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [30]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format("VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO", "VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM", 51.5160797, 0.0594257, "20180605", 500, 100)
url

'https://api.foursquare.com/v2/venues/explore?client_id=VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO&client_secret=VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM&ll=51.5160797,0.0594257&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffc6f11cb7cea4da4536a97'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Beckton',
  'headerFullLocation': 'Beckton, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 51.5205797045, 'lng': 0.06664350286675196},
   'sw': {'lat': 51.5115796955, 'lng': 0.05220789713324805}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e0c524018a89ad010688d2b',
       'name': 'East london Gymnastics Club',
       'location': {'lat': 51.514106774737556,
        'lng': 0.060155068624099396,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.514106774737556,
          'lng': 0.0601

Lets define the get_category_type function

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\JA\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,East london Gymnastics Club,Gym / Fitness Center,51.514107,0.060155
1,Home Bargains,Discount Store,51.516790,0.062967
2,Lidl,Supermarket,51.515982,0.054794
3,Lituanica,Grocery Store,51.516442,0.062927
4,Premier Inn London Beckton,Hotel,51.515151,0.061013


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


## 2 Explore Neighborhoods in Newham

#### Now we create a function to repeat the same process to all the neighborhoods in Manhattan

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called newham_venues.

In [36]:
newham_venues = getNearbyVenues(names=df_coord['Neighborhood'],
                                   latitudes=df_coord['Latitude'],
                                   longitudes=df_coord['Longitude']
                                  )

Beckton
Canning Town
Cyprus
East Ham
East Village
Forest Gate
Manor Park
Maryland
North Woolwich
Plashet
Silvertown
Stratford
Upton Park
Wallend
West Ham


### Let's check the size of the resulting dataframe

In [37]:
print(newham_venues.shape)
newham_venues.head()

(258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckton,51.51608,0.059426,East london Gymnastics Club,51.514107,0.060155,Gym / Fitness Center
1,Beckton,51.51608,0.059426,Home Bargains,51.516790,0.062967,Discount Store
2,Beckton,51.51608,0.059426,Lidl,51.515982,0.054794,Supermarket
3,Beckton,51.51608,0.059426,Lituanica,51.516442,0.062927,Grocery Store
4,Beckton,51.51608,0.059426,Premier Inn London Beckton,51.515151,0.061013,Hotel


### Let's check how many venues were returned for each neighborhood

In [38]:
newham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beckton,13,13,13,13,13,13
Canning Town,31,31,31,31,31,31
Cyprus,4,4,4,4,4,4
East Ham,15,15,15,15,15,15
East Village,43,43,43,43,43,43
Forest Gate,11,11,11,11,11,11
Manor Park,16,16,16,16,16,16
Maryland,24,24,24,24,24,24
North Woolwich,6,6,6,6,6,6


### Let's find out how many unique categories can be curated from all the returned venues

In [39]:
print('There are {} uniques categories.'.format(len(newham_venues['Venue Category'].unique())))

There are 101 uniques categories.


## 3. Analyze Each Neighborhood

#### Here we apply one hot encoding to all the venues. So now the number of columns becomes 101

In [40]:
# one hot encoding
newham_onehot = pd.get_dummies(newham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newham_onehot['Neighborhood'] = newham_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newham_onehot.columns[-1]] + list(newham_onehot.columns[:-1])
newham_onehot = newham_onehot[fixed_columns]

newham_onehot.head()

,Neighborhood,Airport Terminal,Art Museum,Athletics & Sports,Bakery,Bar,Beer Bar,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Dam,Dance Studio,Department Store,Dessert Shop,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Service,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Furniture / Home Store,Gastropub,Gelato Shop,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Jazz Club,Kitchen Supply Store,Laser Tag,Lawyer,Light Rail Station,Liquor Store,Market,Metro Station,Modern European Restaurant,Movie Theater,Museum,Nature Preserve,Park,Pharmacy,Pier,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Restaurant,Rugby Pitch,Sandwich Place,Scenic Lookout,Sculpture Garden,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront
0,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Beckton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### And let's examine the new dataframe size.

In [41]:
newham_onehot.shape

(258, 102)

In [42]:
len((newham_onehot[newham_onehot["Shopping Plaza"] > 0]))

1

In [43]:
len((newham_onehot[newham_onehot["Shopping Mall"] > 0]))

1

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
grouped_newham = newham_onehot.groupby('Neighborhood').mean().reset_index()
grouped_newham

,Neighborhood,Airport Terminal,Art Museum,Athletics & Sports,Bakery,Bar,Beer Bar,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Dam,Dance Studio,Department Store,Dessert Shop,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Service,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Furniture / Home Store,Gastropub,Gelato Shop,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Jazz Club,Kitchen Supply Store,Laser Tag,Lawyer,Light Rail Station,Liquor Store,Market,Metro Station,Modern European Restaurant,Movie Theater,Museum,Nature Preserve,Park,Pharmacy,Pier,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Restaurant,Rugby Pitch,Sandwich Place,Scenic Lookout,Sculpture Garden,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront
0,Beckton,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.0000,0.076923,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,Canning Town,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.032258,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.096774,0.000000,0.000000,0.000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.032258,0.032258,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.064516,0.032258,0.032258,0.000000,0.000,0.000000,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.032258,0.0000,0.000000,0.000000,0.000,0.032258,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.064516,0.000000,0.000000,0.032258,0.16129,0.000000,0.000000,0.000000,0.000000,0.0,0.032258,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.032258,0.000000,0.000,0.000000,0.000000
2,Cyprus,0.000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000

### Let's print each neighborhood along with the top 10 most common venues

In [45]:
top_venues = 10

for hood in grouped_newham['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped_newham[grouped_newham['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Beckton----
                    venue  freq
0             Supermarket  0.15
1           Grocery Store  0.08
2             Bus Station  0.08
3  Furniture / Home Store  0.08
4          Discount Store  0.08
5          Shopping Plaza  0.08
6            Soccer Field  0.08
7          Clothing Store  0.08
8    Gym / Fitness Center  0.08
9                     Pub  0.08


----Canning Town----
                  venue  freq
0              Platform  0.16
1           Coffee Shop  0.10
2                   Gym  0.06
3                 Hotel  0.06
4                  Park  0.06
5           Pizza Place  0.03
6     Fish & Chips Shop  0.03
7  Fast Food Restaurant  0.03
8        Sandwich Place  0.03
9    Light Rail Station  0.03


----Cyprus----
                   venue  freq
0          Grocery Store  0.25
1     Athletics & Sports  0.25
2     Light Rail Station  0.25
3            Coffee Shop  0.25
4        Nature Preserve  0.00
5             Restaurant  0.00
6                    Pub  0.00
7  Portuguese 

### let's write a function to sort the venues in descending order.

In [46]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [47]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sorted_neighborhoods_venues = pd.DataFrame(columns=columns)
sorted_neighborhoods_venues['Neighborhood'] = grouped_newham['Neighborhood']

for ind in np.arange(grouped_newham.shape[0]):
    sorted_neighborhoods_venues.iloc[ind, 1:] = return_most_common_venues(grouped_newham.iloc[ind, :], top_venues)

sorted_neighborhoods_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Supermarket,Light Rail Station,Pub,Clothing Store,Shopping Plaza,Soccer Field,Furniture / Home Store,Discount Store,Bus Station,Hotel
1,Canning Town,Platform,Coffee Shop,Hotel,Park,Gym,Gym Pool,Café,Dance Studio,Sandwich Place,Fast Food Restaurant
2,Cyprus,Athletics & Sports,Coffee Shop,Light Rail Station,Grocery Store,Waterfront,Gelato Shop,Electronics Store,English Restaurant,Event Service,Fast Food Restaurant
3,East Ham,Indian Restaurant,Supermarket,Gym Pool,Warehouse Store,Grocery Store,Fast Food Restaurant,Electronics Store,Park,Sandwich Place,Coffee Shop
4,East Village,Café,Coffee Shop,Ice Cream Shop,Italian Restaurant,Park,Modern European Restaurant,Clothing Store,Pub,Creperie,Pizza Place


## 4. Cluster Neighborhoods

##### Lets k-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 5

grouped_newham_clustering = grouped_newham.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_newham_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 3, 0, 0, 4, 3])

In [49]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
# add clustering labels
sorted_neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)

newham_merged = df_coord

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
newham_merged = newham_merged.join(sorted_neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')

newham_merged.head() # check the last columns!

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Newham,51.516080,0.059426,0,Supermarket,Light Rail Station,Pub,Clothing Store,Shopping Plaza,Soccer Field,Furniture / Home Store,Discount Store,Bus Station,Hotel
1,Canning Town,Newham,51.513989,0.008299,0,Platform,Coffee Shop,Hotel,Park,Gym,Gym Pool,Café,Dance Studio,Sandwich Place,Fast Food Restaurant
2,Cyprus,Newham,51.508478,0.063969,1,Athletics & Sports,Coffee Shop,Light Rail Station,Grocery Store,Waterfront,Gelato Shop,Electronics Store,English Restaurant,Event Service,Fast Food Restaurant
3,East Ham,Newham,51.532963,0.055320,0,Indian Restaurant,Supermarket,Gym Pool,Warehouse Store,Grocery Store,Fast Food Restaurant,Electronics Store,Park,Sandwich Place,Coffee Shop
4,East Village,Newham,51.548108,-0.009177,0,Café,Coffee Shop,Ice Cream Shop,Italian Restaurant,Park,Modern European Restaurant,Clothing Store,Pub,Creperie,Pizza Place


### Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[51.5460532, 0.0059223], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newham_merged['Latitude'], newham_merged['Longitude'], newham_merged['Neighborhood'], newham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [52]:
newham_merged.loc[newham_merged['Cluster Labels']== 0, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckton,Supermarket,Light Rail Station,Pub,Clothing Store,Shopping Plaza,Soccer Field,Furniture / Home Store,Discount Store,Bus Station,Hotel
1,Canning Town,Platform,Coffee Shop,Hotel,Park,Gym,Gym Pool,Café,Dance Studio,Sandwich Place,Fast Food Restaurant
3,East Ham,Indian Restaurant,Supermarket,Gym Pool,Warehouse Store,Grocery Store,Fast Food Restaurant,Electronics Store,Park,Sandwich Place,Coffee Shop
4,East Village,Café,Coffee Shop,Ice Cream Shop,Italian Restaurant,Park,Modern European Restaurant,Clothing Store,Pub,Creperie,Pizza Place
6,Manor Park,Vietnamese Restaurant,Café,Grocery Store,Bus Stop,Chinese Restaurant,Sporting Goods Shop,Supermarket,Gastropub,Coffee Shop,Skating Rink
7,Maryland,Pub,Hotel,Bus Stop,Sports Bar,Indie Movie Theater,Indie Theater,Grocery Store,General Entertainment,Liquor Store,Furniture / Home Store
10,Silvertown,Airport Terminal,Hotel,Dam,Gym,Theater,Breakfast Spot,Park,Light Rail Station,Gastropub,Electronics Store
11,Stratford,Hotel,Restaurant,Theater,Gastropub,History Museum,Italian Restaurant,Coffee Shop,Café,Tea Room,Gift Shop
14,West Ham,Platform,Café,Rugby Pitch,Trail,Park,Fish & Chips Shop,Bus Station,Waterfront,Furniture / Home Store,Eastern European Restaurant


In [53]:
newham_merged.loc[newham_merged['Cluster Labels']== 1, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cyprus,Athletics & Sports,Coffee Shop,Light Rail Station,Grocery Store,Waterfront,Gelato Shop,Electronics Store,English Restaurant,Event Service,Fast Food Restaurant


In [54]:
newham_merged.loc[newham_merged['Cluster Labels']== 2, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Wallend,Hotel,Hardware Store,Event Service,Lawyer,Coffee Shop,Supermarket,Waterfront,Gastropub,Eastern European Restaurant,Electronics Store


In [55]:
newham_merged.loc[newham_merged['Cluster Labels']== 3, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Forest Gate,Grocery Store,Fast Food Restaurant,Bus Stop,Train Station,Fish & Chips Shop,Café,Pub,Bakery,Gym Pool,Furniture / Home Store
9,Plashet,Grocery Store,Indian Restaurant,Shopping Mall,Convenience Store,Bus Stop,Harbor / Marina,Gym Pool,Eastern European Restaurant,Electronics Store,English Restaurant
12,Upton Park,Grocery Store,Bus Stop,Pharmacy,Market,Convenience Store,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Pub


In [56]:
newham_merged.loc[newham_merged['Cluster Labels']== 4, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North Woolwich,Waterfront,Harbor / Marina,Tunnel,Pier,Boat or Ferry,Light Rail Station,Gastropub,Eastern European Restaurant,Electronics Store,English Restaurant


# Results

### Let's examine the 5 clusters

Cluster 0 and Cluster 3 has got neighborhoods with at least one Shopping Plazza which is the same as Shopping Mall

Cluster 0: This cluster has got Shopping Plazza in Beckton neighborhood.

Cluster 3: This cluster has got a Shopping Mall in Plashet neighborhood

But cluster 1, cluster 2 and cluster 4 results shows neighborhoods without shopping plazza or shopping mall present in those clusters.

# Conclusion

I would recomend the prospective project developers to think about opening shopping malls or shopping plazzas in neighborhoods where there is non present. Another benefit of capitalizing in these neighborhoods is because their will be no competition from existing malls, if not very little. 